In [5]:
import os
import sys
import random
import warnings
import time

import numpy as np
import pandas as pd
import cv2

import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

import tensorflow as tf

# Set some parameters
IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 3
TRAIN_PATH = 'G:/Kaggle_data/Kaggle_bowl/stage1_train/'
TEST_PATH = 'G:/Kaggle_data/Kaggle_bowl/stage1_test/'

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 42
random.seed = seed
np.random.seed = seed
time = time.strftime("%Y-%m-%d", time.gmtime())

smooth = 1

# Get train and test IDs
train_ids = next(os.walk(TRAIN_PATH))[1]
test_ids = next(os.walk(TEST_PATH))[1]

# Get and resize train images and masks
X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
print('Getting and resizing train images and masks ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
    path = TRAIN_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_train[n] = img
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    for mask_file in next(os.walk(path + '/masks/'))[2]:
        mask_ = imread(path + '/masks/' + mask_file)
        mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                      preserve_range=True), axis=-1)
        mask = np.maximum(mask, mask_)
    Y_train[n] = mask
    
#def scale_img_canals(an_img):
for i in range(IMG_CHANNELS):
    canal = img[:,:,i]
    canal = canal - canal.min()
    canalmax = canal.max()
    if canalmax > 0:
        factor = 255/canalmax
        canal = (canal * factor).astype(int)
    img[:,:,i] = canal

# Get and resize test images
X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
sizes_test = []
print('Getting and resizing test images ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = TEST_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img
    
for i in range(IMG_CHANNELS):
    canal = img[:,:,i]
    canal = canal - canal.min()
    canalmax = canal.max()
    if canalmax > 0:
        factor = 255/canalmax
        canal = (canal * factor).astype(int)
    img[:,:,i] = canal

print('Done!')

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)

def get_more_images(imgs):
    
    more_images = []
    vert_flip_imgs = []
    hori_flip_imgs = []
    rotated_images_pos90 = []
    rotated_images_neg90 = []
    
    for i in range(0,imgs.shape[0]):
        a=imgs[i,:,:,0]
        b=imgs[i,:,:,1]
        c=imgs[i,:,:,2]
        
#         IMG_HEIGHT = imgs.shape[0]
#         IMG_WIDTH = imgs.shape[1]
    
        av=cv2.flip(a,1)
        ah=cv2.flip(a,0)
        bv=cv2.flip(b,1)
        bh=cv2.flip(b,0)
        cv=cv2.flip(c,1)
        ch=cv2.flip(c,0)
        
        arp90=cv2.getRotationMatrix2D((IMG_WIDTH/2,IMG_HEIGHT/2),90,1)
        arp90_r = cv2.warpAffine(a,arp90,(IMG_WIDTH,IMG_HEIGHT))
        
        arn90=cv2.getRotationMatrix2D((IMG_WIDTH/2,IMG_HEIGHT/2),270,1)
        arn90_r = cv2.warpAffine(a,arn90,(IMG_WIDTH,IMG_HEIGHT))
        
        brp90=cv2.getRotationMatrix2D((IMG_WIDTH/2,IMG_HEIGHT/2),90,1)
        brp90_r = cv2.warpAffine(a,brp90,(IMG_WIDTH,IMG_HEIGHT))
        
        brn90=cv2.getRotationMatrix2D((IMG_WIDTH/2,IMG_HEIGHT/2),270,1)
        brn90_r = cv2.warpAffine(a,brn90,(IMG_WIDTH,IMG_HEIGHT))
        
        crp90=cv2.getRotationMatrix2D((IMG_WIDTH/2,IMG_HEIGHT/2),90,1)
        crp90_r = cv2.warpAffine(a,crp90,(IMG_WIDTH,IMG_HEIGHT))
        
        crn90=cv2.getRotationMatrix2D((IMG_WIDTH/2,IMG_HEIGHT/2),270,1)
        crn90_r = cv2.warpAffine(a,crn90,(IMG_WIDTH,IMG_HEIGHT))
        
        vert_flip_imgs.append(np.dstack((av, bv, cv)))
        hori_flip_imgs.append(np.dstack((ah, bh, ch)))
        rotated_images_pos90.append(np.dstack((arp90_r, brp90_r, crp90_r)))
        rotated_images_neg90.append(np.dstack((arn90_r, brn90_r, crn90_r)))
    
    v = np.array(vert_flip_imgs)
    h = np.array(hori_flip_imgs)
    rp = np.array(rotated_images_pos90)
    rn = np.array(rotated_images_neg90)
    
    print(v)
#     print('h shape \n' + h)
    print(rp)
#     print('rn shape \n' + rn)
    more_images = np.concatenate((imgs,v,h,rp,rn))
    
    return more_images

def duplicate_labels(labels):
    more_images = []
    vert_flip_imgs = []
    hori_flip_imgs = []
    rotated_images_pos90 = []
    rotated_images_neg90 = []
    for i in range(0,labels.shape[0]):
        
#         IMG_HEIGHT = labels.shape[0]
#         IMG_WIDTH = labels.shape[1]
        
        a=labels[i,:,:,0]

        av=cv2.flip(a,1)
        ah=cv2.flip(a,0)
        
        arp90=cv2.getRotationMatrix2D((IMG_WIDTH/2,IMG_HEIGHT/2),90,1)
        arp90_r = cv2.warpAffine(a,arp90,(IMG_WIDTH,IMG_HEIGHT))
        
        arn90=cv2.getRotationMatrix2D((IMG_WIDTH/2,IMG_HEIGHT/2),-90,1)
        arn90_r = cv2.warpAffine(a,arn90,(IMG_WIDTH,IMG_HEIGHT))
        
        vert_flip_imgs.append(av.reshape(IMG_WIDTH,IMG_WIDTH,1))
        hori_flip_imgs.append(ah.reshape(IMG_WIDTH,IMG_WIDTH,1))
        rotated_images_pos90.append(arp90_r.reshape(IMG_WIDTH,IMG_WIDTH,1))
        rotated_images_neg90.append(arn90_r.reshape(IMG_WIDTH,IMG_WIDTH,1))
        
    v = np.array(vert_flip_imgs)
    h = np.array(hori_flip_imgs)
    rp = np.array(rotated_images_pos90)
    rn = np.array(rotated_images_neg90)
    duplicate_labels = np.concatenate((labels,v,h,rp,rn))
    return duplicate_labels

X_validation = X_train[-50:,...]
Y_validation = Y_train[-50:]
X_train = X_train[0:-50,...]
Y_train = Y_train[0:-50]
print("Validation Set of Size "+str(Y_validation.shape[0])+" Separated")
Y_train.dtype = np.uint8
X_train = get_more_images(X_train)
Y_train = duplicate_labels(Y_train)
print (X_train.shape)
print (Y_train.shape)
print (X_validation.shape)
print (Y_validation.shape)
print("Data Rotation and Flipping Complete")
print (X_train.shape)
print (Y_train.shape)
print (X_validation.shape)
print (Y_validation.shape)
X_train = np.concatenate((X_train,X_validation),axis=0)
Y_train = np.concatenate((Y_train,Y_validation),axis=0)
np.savez_compressed(file='train.npz',X=X_train,Y=Y_train)
np.savez_compressed(file='test.npz',X=X_test)

# npz = np.load('G:/Kaggle_data/Kaggle_bowl/train.npz') 
# X_train2 = npz['X'] 
# Y_train2 = npz['Y']
# X_test2 = np.load('G:/Kaggle_data/Kaggle_bowl/test.npz')['X']

# Check if training data looks all right
# ix = random.randint(0, len(train_ids))
# imshow(X_train[ix])
# plt.show()
# imshow(np.squeeze(Y_train[ix]))
# plt.show()

# Define IoU metric
"""def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)"""

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

# Build U-Net model
inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = Lambda(lambda x: x / 255) (inputs)

c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (s)
c1 = Dropout(0.1) (c1)
c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.1) (c2)
c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
c3 = Dropout(0.2) (c3)
c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
c4 = Dropout(0.2) (c4)
c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
c5 = Dropout(0.3) (c5)
c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
c6 = Dropout(0.2) (c6)
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
c7 = Dropout(0.2) (c7)
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
c8 = Dropout(0.1) (c8)
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
c9 = Dropout(0.1) (c9)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef])
model.summary()

# Fit model
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint('model-dsbowl2018-1_+img_rotated.h5', verbose=1, save_best_only=True)
results = model.fit(X_train, Y_train, validation_split=0.2, batch_size=16, epochs=50, 
                    callbacks=[earlystopper, checkpointer])
# results2 = model.fit(X_train2, Y_train2, validation_split=0.1, batch_size=16, epochs=50, 
#                     callbacks=[earlystopper, checkpointer])

# Predict on train, val and test
model = load_model('model-dsbowl2018-1_+img_rotated.h5', custom_objects={'dice_coef': dice_coef})
preds_train = model.predict(X_train[:int(X_train.shape[0]*0.9)], verbose=1)
preds_val = model.predict(X_train[int(X_train.shape[0]*0.9):], verbose=1)
preds_test = model.predict(X_test, verbose=1)

# preds_train2 = model.predict(X_train2[:int(X_train2.shape[0]*0.9)], verbose=1)
# preds_val2 = model.predict(X_train2[int(X_train2.shape[0]*0.9):], verbose=1)
# preds_test2 = model.predict(X_test2, verbose=1)

# Threshold predictions
preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

# preds_train_t2 = (preds_train2 > 0.5).astype(np.uint8)
# preds_val_t2 = (preds_val2 > 0.5).astype(np.uint8)
# preds_test_t2 = (preds_test2 > 0.5).astype(np.uint8)

# Create list of upsampled test masks
preds_test_upsampled = []
for i in range(len(preds_test)):
    preds_test_upsampled.append(resize(np.squeeze(preds_test[i]), 
                                       (sizes_test[i][0], sizes_test[i][1]), 
                                       mode='constant', preserve_range=True))
# for i in range(len(preds_test2)):
#     preds_test_upsampled.append(resize(np.squeeze(preds_test2[i]), 
#                                        (sizes_test[i][0], sizes_test[i][1]), 
#                                        mode='constant', preserve_range=True))
    
# Perform a sanity check on some random training samples
# ix = random.randint(0, len(preds_train_t))
# imshow(X_train[ix])
# plt.show()
# imshow(np.squeeze(Y_train[ix]))
# plt.show()
# imshow(np.squeeze(preds_train_t[ix]))
# plt.show()

# Perform a sanity check on some random validation samples
# ix = random.randint(0, len(preds_val_t))
# imshow(X_train[int(X_train.shape[0]*0.9):][ix])
# plt.show()
# imshow(np.squeeze(Y_train[int(Y_train.shape[0]*0.9):][ix]))
# plt.show()
# imshow(np.squeeze(preds_val_t[ix]))
# plt.show()

# Run-length encoding stolen from https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
def rle_encoding(x):
    dots = np.where(x.T.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

def prob_to_rles(x, cutoff=0.5):
    lab_img = label(x > cutoff)
    for i in range(1, lab_img.max() + 1):
        yield rle_encoding(lab_img == i)
        
new_test_ids = []
rles = []
for n, id_ in enumerate(test_ids):
    rle = list(prob_to_rles(preds_test_upsampled[n]))
    rles.extend(rle)
    new_test_ids.extend([id_] * len(rle))
    
# Create submission DataFrame

sub = pd.DataFrame()
sub['ImageId'] = new_test_ids
sub['EncodedPixels'] = pd.Series(rles).apply(lambda x: ' '.join(str(y) for y in x))
sub.to_csv('sub-dsbowl2018-1_%s_256_scale+flip_rotate.csv' % time, index=False)



Using TensorFlow backend.


ImportError: cannot import name 'self_check'

65/65 [==============================] - ETA:  - ETA:  - 11s 169ms/step
